# SmartPave Analytics: Data Exploration

## Overview
This notebook explores the pavement condition and maintenance data for 16,000 miles of roadway infrastructure.

## Objectives
- Load and examine the dataset structure
- Understand data quality and completeness
- Create initial visualizations
- Identify patterns and trends
- Prepare for feature engineering


## Setup and Imports


In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries imported successfully!")


## Data Loading


In [ ]:
# Set database context and load datasets from Snowflake tables
print("Loading datasets from Snowflake...")

# Connect to Snowflake and set context
from snowflake.snowpark.context import get_active_session
session = get_active_session()
session.sql("USE DATABASE DOT_workshop_test").collect()
session.sql("USE SCHEMA smartpave_analytics").collect()

# Road network data
roads_df = session.sql("SELECT * FROM DOT_workshop_test.smartpave_analytics.road_network").to_pandas()
print(f"Road network: {len(roads_df):,} segments")
print(f"Road network columns: {list(roads_df.columns)}")

# Pavement condition data
condition_df = session.sql("SELECT * FROM DOT_workshop_test.smartpave_analytics.pavement_condition").to_pandas()
print(f"Pavement condition: {len(condition_df):,} records")
print(f"Pavement condition columns: {list(condition_df.columns)}")

# Check if date column exists and convert if it does
if 'date' in condition_df.columns:
    condition_df['date'] = pd.to_datetime(condition_df['date'])
    print("✅ Date column converted to datetime")
else:
    print("⚠️ No 'date' column found in pavement_condition table")
    # Check what date-related columns exist
    date_cols = [col for col in condition_df.columns if 'date' in col.lower()]
    print(f"Date-related columns found: {date_cols}")

# Maintenance records
maintenance_df = session.sql("SELECT * FROM DOT_workshop_test.smartpave_analytics.maintenance_records").to_pandas()
print(f"Maintenance records: {len(maintenance_df):,} records")
print(f"Maintenance records columns: {list(maintenance_df.columns)}")

# Check if date column exists and convert if it does
if 'date' in maintenance_df.columns:
    maintenance_df['date'] = pd.to_datetime(maintenance_df['date'])
    print("✅ Date column converted to datetime")
else:
    print("⚠️ No 'date' column found in maintenance_records table")

# Traffic data
traffic_df = session.sql("SELECT * FROM DOT_workshop_test.smartpave_analytics.traffic_data").to_pandas()
print(f"Traffic data: {len(traffic_df):,} records")
print(f"Traffic data columns: {list(traffic_df.columns)}")

print("\n✅ All datasets loaded successfully from Snowflake!")


In [ ]:
# Dataset Overview and Basic Statistics
print("="*60)
print("DATASET OVERVIEW")
print("="*60)

# Basic statistics
print(f"📊 Dataset Summary:")
print(f"   • Road segments: {len(roads_df):,}")
print(f"   • Pavement records: {len(condition_df):,}")
print(f"   • Maintenance records: {len(maintenance_df):,}")
print(f"   • Traffic records: {len(traffic_df):,}")

# Date ranges (only if date columns exist)
print(f"\n📅 Date Ranges:")
if 'date' in condition_df.columns:
    print(f"   • Pavement data: {condition_df['date'].min().strftime('%Y-%m-%d')} to {condition_df['date'].max().strftime('%Y-%m-%d')}")
else:
    print("   • Pavement data: No date column available")

if 'date' in maintenance_df.columns:
    print(f"   • Maintenance data: {maintenance_df['date'].min().strftime('%Y-%m-%d')} to {maintenance_df['date'].max().strftime('%Y-%m-%d')}")
else:
    print("   • Maintenance data: No date column available")

# Road types distribution
print(f"\n🛣️ Road Types:")
if 'road_type' in roads_df.columns:
    road_type_counts = roads_df['road_type'].value_counts()
    for road_type, count in road_type_counts.items():
        print(f"   • {road_type}: {count:,} segments ({count/len(roads_df)*100:.1f}%)")
else:
    print("   • No road_type column found")

# Condition score statistics
print(f"\n📈 Pavement Condition Statistics:")
if 'condition_score' in condition_df.columns:
    print(f"   • Average condition score: {condition_df['condition_score'].mean():.1f}")
    print(f"   • Median condition score: {condition_df['condition_score'].median():.1f}")
    print(f"   • Best condition: {condition_df['condition_score'].max():.1f}")
    print(f"   • Worst condition: {condition_df['condition_score'].min():.1f}")
else:
    print("   • No condition_score column found")

# Maintenance cost statistics
print(f"\n💰 Maintenance Cost Statistics:")
if 'cost' in maintenance_df.columns:
    print(f"   • Total maintenance cost: ${maintenance_df['cost'].sum():,.0f}")
    print(f"   • Average repair cost: ${maintenance_df['cost'].mean():,.0f}")
    print(f"   • Most expensive repair: ${maintenance_df['cost'].max():,.0f}")
    print(f"   • Total repairs: {len(maintenance_df):,}")
else:
    print("   • No cost column found")


In [ ]:
# Data Quality Analysis
print("="*60)
print("DATA QUALITY ANALYSIS")
print("="*60)

# Check for missing values
print("🔍 Missing Values Analysis:")
datasets = {
    'Road Network': roads_df,
    'Pavement Condition': condition_df,
    'Maintenance Records': maintenance_df,
    'Traffic Data': traffic_df
}

for name, df in datasets.items():
    missing = df.isnull().sum()
    total = len(df)
    print(f"\n{name}:")
    if missing.sum() == 0:
        print("   ✅ No missing values")
    else:
        for col, missing_count in missing[missing > 0].items():
            pct = (missing_count / total) * 100
            print(f"   ⚠️ {col}: {missing_count:,} missing ({pct:.1f}%)")

# Check for duplicates
print(f"\n🔍 Duplicate Records Analysis:")
for name, df in datasets.items():
    duplicates = df.duplicated().sum()
    print(f"   • {name}: {duplicates:,} duplicate records")

# Check data types
print(f"\n🔍 Data Types Analysis:")
for name, df in datasets.items():
    print(f"\n{name}:")
    for col, dtype in df.dtypes.items():
        print(f"   • {col}: {dtype}")

# Check for outliers in condition scores
print(f"\n🔍 Outlier Analysis (Condition Scores):")
Q1 = condition_df['condition_score'].quantile(0.25)
Q3 = condition_df['condition_score'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers = condition_df[(condition_df['condition_score'] < lower_bound) | 
                       (condition_df['condition_score'] > upper_bound)]
print(f"   • Outliers detected: {len(outliers):,} ({len(outliers)/len(condition_df)*100:.1f}%)")
print(f"   • Normal range: {lower_bound:.1f} to {upper_bound:.1f}")


In [ ]:
# Visualization 1: Condition Score Distribution
plt.figure(figsize=(15, 5))

# Histogram of condition scores
plt.subplot(1, 3, 1)
plt.hist(condition_df['condition_score'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
plt.title('Distribution of Pavement Condition Scores')
plt.xlabel('Condition Score')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

# Box plot by road type
plt.subplot(1, 3, 2)
condition_with_road_type = condition_df.merge(roads_df[['segment_id', 'road_type']], on='segment_id')
road_types = condition_with_road_type['road_type'].unique()
box_data = [condition_with_road_type[condition_with_road_type['road_type'] == rt]['condition_score'].values 
           for rt in road_types]
plt.boxplot(box_data, labels=road_types)
plt.title('Condition Scores by Road Type')
plt.xlabel('Road Type')
plt.ylabel('Condition Score')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Time series of average condition
plt.subplot(1, 3, 3)
monthly_condition = condition_df.groupby(condition_df['date'].dt.to_period('M'))['condition_score'].mean()
plt.plot(monthly_condition.index.astype(str), monthly_condition.values, marker='o', linewidth=2)
plt.title('Average Condition Score Over Time')
plt.xlabel('Month')
plt.ylabel('Average Condition Score')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Visualization 1 Complete: Condition Score Analysis")


In [ ]:
# Visualization 2: Maintenance Cost Analysis
plt.figure(figsize=(15, 5))

# Maintenance cost distribution
plt.subplot(1, 3, 1)
plt.hist(maintenance_df['cost'], bins=50, alpha=0.7, color='lightcoral', edgecolor='black')
plt.title('Distribution of Maintenance Costs')
plt.xlabel('Cost ($)')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

# Cost by repair type
plt.subplot(1, 3, 2)
cost_by_type = maintenance_df.groupby('repair_type')['cost'].mean().sort_values(ascending=False)
cost_by_type.plot(kind='bar', color='lightgreen')
plt.title('Average Cost by Repair Type')
plt.xlabel('Repair Type')
plt.ylabel('Average Cost ($)')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Monthly maintenance spending
plt.subplot(1, 3, 3)
monthly_costs = maintenance_df.groupby(maintenance_df['date'].dt.to_period('M'))['cost'].sum()
plt.plot(monthly_costs.index.astype(str), monthly_costs.values, marker='s', linewidth=2, color='orange')
plt.title('Monthly Maintenance Spending')
plt.xlabel('Month')
plt.ylabel('Total Cost ($)')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Visualization 2 Complete: Maintenance Cost Analysis")


In [ ]:
# Visualization 3: Traffic and Condition Correlation
plt.figure(figsize=(15, 5))

# Traffic volume vs condition score
plt.subplot(1, 3, 1)
condition_with_traffic = condition_df.merge(roads_df[['segment_id', 'traffic_volume']], on='segment_id')
plt.scatter(condition_with_traffic['traffic_volume'], condition_with_traffic['condition_score'], 
           alpha=0.5, s=10, color='purple')
plt.title('Traffic Volume vs Condition Score')
plt.xlabel('Traffic Volume (vehicles/day)')
plt.ylabel('Condition Score')
plt.grid(True, alpha=0.3)

# Correlation heatmap
plt.subplot(1, 3, 2)
correlation_data = condition_with_traffic[['condition_score', 'traffic_volume', 'roughness_index', 
                                         'cracking_percent', 'pothole_count']].corr()
sns.heatmap(correlation_data, annot=True, cmap='coolwarm', center=0, square=True)
plt.title('Feature Correlation Matrix')
plt.tight_layout()

# Condition degradation over time by road type
plt.subplot(1, 3, 3)
for road_type in condition_with_road_type['road_type'].unique():
    road_data = condition_with_road_type[condition_with_road_type['road_type'] == road_type]
    monthly_condition = road_data.groupby(road_data['date'].dt.to_period('M'))['condition_score'].mean()
    plt.plot(monthly_condition.index.astype(str), monthly_condition.values, 
             marker='o', label=road_type, linewidth=2)
plt.title('Condition Trends by Road Type')
plt.xlabel('Month')
plt.ylabel('Average Condition Score')
plt.legend()
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Visualization 3 Complete: Traffic and Condition Analysis")


In [ ]:
# Summary and Key Insights
print("="*60)
print("KEY INSIGHTS FROM DATA EXPLORATION")
print("="*60)

# Calculate key metrics
total_road_miles = roads_df['segment_length_miles'].sum()
total_maintenance_cost = maintenance_df['cost'].sum()
avg_condition = condition_df['condition_score'].mean()
worst_condition = condition_df['condition_score'].min()
best_condition = condition_df['condition_score'].max()

print(f"📊 Infrastructure Overview:")
print(f"   • Total road network: {total_road_miles:,.0f} miles")
print(f"   • Total maintenance investment: ${total_maintenance_cost:,.0f}")
print(f"   • Average condition score: {avg_condition:.1f}/100")
print(f"   • Condition range: {worst_condition:.1f} to {best_condition:.1f}")

# Identify problem areas
poor_condition = condition_df[condition_df['condition_score'] < 30]
high_traffic_poor_condition = condition_with_traffic[
    (condition_with_traffic['condition_score'] < 30) & 
    (condition_with_traffic['traffic_volume'] > 25000)
]

print(f"\n⚠️ Problem Areas Identified:")
print(f"   • Segments in poor condition (<30): {len(poor_condition):,}")
print(f"   • High-traffic poor condition: {len(high_traffic_poor_condition):,}")
print(f"   • Percentage needing attention: {len(poor_condition)/len(condition_df)*100:.1f}%")

# Cost analysis
cost_per_mile = total_maintenance_cost / total_road_miles
print(f"\n💰 Cost Analysis:")
print(f"   • Average cost per mile: ${cost_per_mile:,.0f}")
print(f"   • Most expensive repair type: {maintenance_df.groupby('repair_type')['cost'].mean().idxmax()}")
print(f"   • Average repair cost: ${maintenance_df['cost'].mean():,.0f}")

# Recommendations
print(f"\n🎯 Initial Recommendations:")
print(f"   • Focus on {len(high_traffic_poor_condition):,} high-traffic segments in poor condition")
print(f"   • Prioritize {poor_condition['segment_id'].nunique():,} unique segments needing repair")
print(f"   • Consider preventive maintenance for segments scoring 30-50")
print(f"   • Monitor {roads_df['road_type'].value_counts().idxmax()} roads (most common type)")

print(f"\n✅ Data exploration complete! Ready for feature engineering.")
